## 코드
### 아래에 정리 후 삭제

# 1. Library Import

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# 2. use Gpu or Cpu

In [ ]:
# gpu를 사용할 수 있는지 확인해주는 API
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")
 

# 3. Hyperparmeter

In [ ]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5
 

# 4. Data load

In [ ]:
# MNIST dataset download API for train
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
# MNIST dataset download API for test
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

# dataset을 batchsize로 나누어주고 shuffle을 하거나 여러가지 customize가능한 API
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
 
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape
 

# 5. Model define

In [ ]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input_channel = 1, output_channel = 10, kernel size = 5, stribe = 1인 convolution연산 api
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # random하게 2d 입력에 대해서 channel을 끊어주는 연산을 해주는 API
        self.mp = nn.MaxPool2d(2) # kernel_size=(2,2), stride=2, sliding window방식으로 max값으로 축소해주는 API
        self.fc1 = nn.Linear(320,100)  # input=320->output=100 인 Perecptron Layer API
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # max(0,x) 연산을 해주는 RELU API
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x)

# 6. Train

In [ ]:
 
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)   # CrossEntropyLoss을 연산해주는 API
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)   # Adam optimizer API
 
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        #모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : 이전 grad영향을 없애기 위해서
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        # grad을 계산하는 API
        cost.backward()
        # 계산된 grad을 가지고 파라미터값을 수정해주는 API
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# 7.  Test



In [ ]:
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')